In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data/Robust_Scaler_result.csv',encoding='cp949')

In [3]:

from sklearn.model_selection import train_test_split

In [4]:
features = data[[ 'CBSCORE', 'CREDITOTAMT', 'YSALEAMT',
       'ESTMM', 'IMSAAMT', 'IMJUAMT',  'BUJUAMT',
       'BU1TOTAMT', 'GAMT', 'KOSPI', '환율', 'GDP', 
       '실업률', '물가지수', '국고채', '금리', '유가등락률',
       '소비자심리지수','ONEHOT']] #GKIND 갖고 가나요?.. 아뇨..ㅋㅋㅋ
act = data['ACTCD'] 

In [5]:

train_x, test_x,train_y, test_y = train_test_split(features, act,
stratify=act,train_size=0.7,test_size=0.3,random_state=1)

print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(27381, 19) (11736, 19) (27381,) (11736,)


In [6]:
'''
1. ENN
2. ADASYN
3. BORDERLINE SMOTE
4. Tomeklinks
5 SOTEENN
6.SMOTETOMEK
'''
from imblearn.over_sampling import BorderlineSMOTE,ADASYN
from imblearn.combine import SMOTEENN,SMOTETomek
from imblearn.under_sampling import EditedNearestNeighbours, TomekLinks
data_sets={}
enn = EditedNearestNeighbours()
ada = ADASYN(random_state=42)
smte = SMOTEENN(random_state=42)
smtt = SMOTETomek(random_state=42)
tk = TomekLinks()
sm = BorderlineSMOTE(random_state=42)
enn_under_train_x,enn_under_train_y = enn.fit_resample(train_x,train_y)
ada_over_train_x, ada_over_train_y = ada.fit_resample(train_x,train_y)
smte_combine_train_x,smte_combine_train_y = smte.fit_resample(train_x,train_y)
smtt_combine_train_x,smtt_combine_train_y = smtt.fit_resample(train_x,train_y)
tk_under_train_x,tk_under_train_y = tk.fit_resample(train_x,train_y)
sm_over_train_x, sm_over_train_y = sm.fit_resample(train_x,train_y)
data_sets['enn']=[enn_under_train_x,enn_under_train_y]
data_sets['ada']=[ada_over_train_x,ada_over_train_y]
data_sets['smte']=[smte_combine_train_x,smte_combine_train_y]
data_sets['smtt']=[smtt_combine_train_x,smtt_combine_train_y]
data_sets['tk']=[tk_under_train_x,tk_under_train_y]
data_sets['sm']=[sm_over_train_x,sm_over_train_y]
'''
over_train_x : 학습용 독립변수
over_train_y : 학습용 종속변수
'''
keys_list= data_sets.keys()
accuracies = []
matrix_kd_tree_uniform_BOST = []

In [7]:
keys_list

dict_keys(['enn', 'ada', 'smte', 'smtt', 'tk', 'sm'])

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import precision_score,recall_score,f1_score,roc_auc_score,confusion_matrix, classification_report


In [9]:
for i in keys_list:
    model = RandomForestClassifier(n_estimators=200,max_depth=50,max_features=1)
    model.fit(data_sets[i][0], data_sets[i][1])
                            
                    # 결정계수
    relation_square = model.score(data_sets[i][0], data_sets[i][1])
    print('결정계수 : ', relation_square)
                    
                    # 예측
    pred_y = model.predict(test_x)
    pr = precision_score(y_true=test_y,y_pred=pred_y)
    r = recall_score(y_true=test_y,y_pred=pred_y)
    f1 = f1_score(y_true=test_y,y_pred=pred_y)
    rca = roc_auc_score(y_true=test_y,y_score=pred_y)
    cfm = confusion_matrix(y_true=test_y,y_pred=pred_y)
    matrix_kd_tree_uniform_BOST.append((i,pr,r,f1,rca,model.score(test_x, test_y),cfm))
                # 정확도 확인
    print(i)
    print('정확도 : ', metrics.accuracy_score(test_y, pred_y))
    print(classification_report(test_y, pred_y, target_names=['사고보증', '정상보증']))
    print('\n')

결정계수 :  0.9990679921888869
enn
정확도 :  0.9254430811179277
              precision    recall  f1-score   support

        사고보증       0.93      1.00      0.96     10666
        정상보증       0.83      0.23      0.36      1070

    accuracy                           0.93     11736
   macro avg       0.88      0.61      0.66     11736
weighted avg       0.92      0.93      0.91     11736



결정계수 :  0.9989344591877765
ada
정확도 :  0.9176039536468984
              precision    recall  f1-score   support

        사고보증       0.93      0.98      0.96     10666
        정상보증       0.61      0.26      0.37      1070

    accuracy                           0.92     11736
   macro avg       0.77      0.62      0.66     11736
weighted avg       0.90      0.92      0.90     11736



결정계수 :  1.0
smte
정확도 :  0.8694614860259032
              precision    recall  f1-score   support

        사고보증       0.94      0.92      0.93     10666
        정상보증       0.33      0.40      0.36      1070

    accuracy         

In [10]:
matrix_kd_tree_uniform_BOST

[('enn',
  0.8260869565217391,
  0.2308411214953271,
  0.36084733382030676,
  0.6129829083943914,
  0.9254430811179277,
  array([[10614,    52],
         [  823,   247]], dtype=int64)),
 ('ada',
  0.6122004357298475,
  0.2626168224299065,
  0.3675604970568999,
  0.6229641396979835,
  0.9176039536468984,
  array([[10488,   178],
         [  789,   281]], dtype=int64)),
 ('smte',
  0.3257918552036199,
  0.40373831775700936,
  0.36060100166944914,
  0.6599602895741732,
  0.8694614860259032,
  array([[9772,  894],
         [ 638,  432]], dtype=int64)),
 ('smtt',
  0.5982721382289417,
  0.2588785046728972,
  0.3613829093281148,
  0.6207199573805139,
  0.9165814587593729,
  array([[10480,   186],
         [  793,   277]], dtype=int64)),
 ('tk',
  0.9043824701195219,
  0.21214953271028036,
  0.3436790310370931,
  0.6049496960382454,
  0.9261247443762781,
  array([[10642,    24],
         [  843,   227]], dtype=int64)),
 ('sm',
  0.6386946386946387,
  0.2560747663551402,
  0.3655770513675784,


In [30]:
type(test_y)

pandas.core.series.Series

In [32]:
aaa= test_y.to_frame(name='act')

In [42]:
type(aaa)

pandas.core.frame.DataFrame

In [45]:
ab = aaa[aaa['act']==1]

In [46]:
ab

,act
8618,1
10479,1
23298,1
20234,1
36829,1
...,...
34635,1
7453,1
1610,1
20619,1
